<center><img alt="title" src="./assets/TFT.jpg"/></center>

# <center>Predicting the Best Strategy to Win in Teamfight Tactics</center>

**<center>Jason Dang, Nhat-Nguyen Vo, Wisely Kong</center>**

## <center>Introduction</center>

Teamfight Tactics (TFT) is an auto battler game developed by Riot Games. Auto battlers are strategy based games in which the player is given preparation time in order to assemble a chess-like board with various units in order to fight an opposing player's board. These fights between boards do not receive any additional input from its players, as the units automatically move and attack until the enemy's board has no units.

Since TFT is a rather complex strategy game, there a variety of concepts to understand about the game.

In the game of TFT, there are 8 players per game. The main objective in each game is to be among the last 4 players left alive, as the game considers "Top 4" to be a moral victory. But in order to truly win a game of TFT, a player must obviously be the last player left alive. The main factors that go into a player's game are: their Health, Gold, Level, Units, Augments, and Components/Items.

- Health: Each player starts off with 100 Health. When a player loses a battle, their health is reduced based on how long the game has lasted and how many units are remaining on the board. At 0 Health, the player is defeated.
- Gold: During each preparation phase, the player is given Gold to utilize in order to make their board as strong as possible. Players can spend their Gold on either Levels or Units in order to do so.
- Level: The amount of units a player can place on their board is dependent on their current level. A player at level 3 can only place 3 units, level 4 can only place 4 units, and so on. The player naturally levels up as the game progresses but can also utilize their Gold during the preparation phase to level up faster.
- Units: During the preparation phase before facing against an opposing player's board, the player is presented a "Shop" with 5 Units in which they can utilize their Gold in order to purchase and place on their board for the next fight. The player can choose to use their Gold instead to re-roll the Shop to choose from 5 different choices. The Units in the Shop have three important aspects: Traits, Cost, and Rarity
    - Traits: Units have "Traits" that grant special effects for Unit with the Trait, but these effects can only be activated when a certain number of unique Units (with the Trait) are placed on the board at the same time. The more Units of the same trait, the stronger the effect.
    - Cost: When Units appear in the Shop, they cost a certain amount of Gold to purchase. The amount a Unit costs relates to how powerful the Unit is, where a Unit costing 1 Gold (the lowest amount) is comparatively weaker to a Unit costing 5 Gold (the highest amount). In addition, the Cost of a Unit relates to how often they would appear in a Shop, where a 1 Gold Unit will appear more often than a 5 Gold Unit.
    - Rarity: When first purchased from the Shop, Units starts at "1-Star". When 3 of the same Unit is purchased the Unit upgrades into "2-Star", in which the Unit's strength is increased. When 3 "2-Star" Units (of the same type) are made, the Unit becomes a "3-Star". The likelihood of reaching "3-Star" is more likely for lower Cost Units (1, 2, 3 Cost Units) compared to higher Cost Units (4, 5 Cost Units) due to how often lower Cost Units appear in the Shop.
- Augments: In certain intervals of the game players are presented with 3 options that provides a variety of benefits to the player, by either making certain Traits stronger or being granted multiple 5 Cost Units for free. Players can only have 3 Augments per game.
- Components/Items: In certain intervals of the game, players are granted Components in which they can place on their Units to make them stronger. When two Components are combined, an Item is created that grants special effects for that specific Units. A Unit can only have a maximum of 3 Items / Components and cannot be interchanged once placed. Ideally, the most important Units on a player's board will always receive 3 Items.

For more information on the game, more information can be found [here](https://tftactics.gg/).

<center><img alt="ExampleTFT" src="./assets/TFT_Example.png" title="TFT Concepts in Game"/></center>

<center><img alt="ExampleAugments"  src="./assets/Augments.png" title="Augment Example"/></center>

In this tutorial...

## <center>Data Collection</center>

In order to predict the best strategy in order to win in TFT, we must first decide where we will be retrieving our data. We decided the best source of data would come from the best players in the game, as they are considered the best for a reason: they know how to win. In TFT, the top players at a given time can be located through [this link, showing the leaderboard for the top players in North America](https://lolchess.gg/leaderboards?mode=ranked&region=na). As for the contents of the data, we arbitrarily chose to select the top 100 players and their last 75 games played. Since the leaderboard is constantly changing every few minutes, as players move up and down the leaderboard, we uploaded a compiled dataset to [Kaggle](https://www.kaggle.com/datasets/cmsc320tftproject/cmsc320-tft-challenger-dataset-set-65) to ease the replication process of this tutorial and have to have the data be consistent with what we use. But for clarity sakes, we will showcase the data scraping process.

The first step is gathering the names of those within the Top 100 rankings, based on the leaderboard website. The next step is to utilize the Riot Games API in order to extract the necessary data, that being the most recent 75 games those 100 players have played. In order to access the Riot Games API, you must [create / login](https://developer.riotgames.com/) to a Riot Games account to generate a development API key. Again, downloading the dataset from Kaggle will allow you to avoid having to deal with creating a development API key. Once the names are gathered and an API Key is obtained, the final step would be to compile and tidy the relevant data into a table.

In [1]:
# Import the necessary libraries that will be used for the tutorial
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [3]:
# Obtain the HTML for the leaderboard website in order to selectively obtain the names of those in the Top 100.
leaderboard = requests.get("https://lolchess.gg/leaderboards?mode=ranked&region=na")
# Using the library BeautifulSoup, parse the HTML to obtain the required information easier (the names of the Top 100)
soup = BeautifulSoup(leaderboard.content, 'html.parser')

In [27]:
# Create an array in order to hold the names
playerList = []

# Limit the HTML to show just the relevant sections (the names)
table_body=soup.find('tbody')
rows = table_body.find_all('tr')

# Iterate through the HTML and store specifically the names of the player into the array.
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    # Separate irrelevant text from the player's name with .partition()
    playerList.append(cols[1].partition("\n\n"))

# Need to remove the irrelevant text that resulted from the previous .partition() call
cleanList = []
for currentPlayer in playerList:
    player = (currentPlayer[2].lstrip())
    cleanList.append(player);